In [211]:
import requests as req
newsurl='https://baike.baidu.com/item/Python/407313?fr=aladdin'
res=req.get(newsurl)
print(res.text)

TooManyRedirects: Exceeded 30 redirects.

In [101]:
##将网页镀金BeautifulSoup中
from bs4 import BeautifulSoup as bfs
html_sample='\
<html>\
 <body>\
 <h1 id="title"><a>Hello World</a></h1>\
 <a href="#" class="link">This is link1</a>\
 <a href="# link2" class="link">This is link2</a>\
 </body>\
 </html>'
soup=bfs(html_sample,'html.parser')
print(soup.text)

  Hello World This is link1 This is link2  


In [119]:
#使用select找出含有h1卷标的元素
soup=bfs(html_sample,'lxml')
header=soup.select('h1')
for h in header:
    print(h)
    a = h.select('a')
    print('a[0]:',a[0].text)
print(header)

<h1 id="title"><a>Hello World</a></h1>
a[0]: Hello World
[<h1 id="title"><a>Hello World</a></h1>]


In [109]:
#使用select找出含有a卷标的元素
soup=bfs(html_sample,'lxml')
alink=soup.select('a')
for link in alink:
    print(link.text)

Hello World
This is link1
This is link2


In [16]:
#使用select找出所有id为title的元素（id前面加#）
alink=soup.select('#title')
print(alink)

[<h1 id="title">Hello World</h1>]


In [21]:
#找出所有class为link的元素（class前面加.)
soup=bfs(html_sample,'lxml')
for link in soup.select('.link'):
    print(link)
    print(link.text)

<a class="link" href="#">This is link1</a>
This is link1
<a class="link" href="# link2">This is link2</a>
This is link2


In [19]:
#找出所有a tag的href链接
alinks=soup.select('a')
for link in alinks:
    print(link['href'])

#
# link2


In [40]:
soup_t=bfs(res.text,'lxml')
for ele in soup_t.select('.news-list'):
    #print(ele.text)
    print(ele.select('a')[0].text,ele.select('a')[0]['href'])

90后都工作了，“反向春运”高峰也就不稀奇了 https://new.qq.com/omn/20190111/20190111A18R8U.html


## 爬取澎湃新闻 简单爬虫

In [207]:
### spiderMain

import requests
from bs4 import BeautifulSoup as bfs

## url 管理器
class UrlManager(object):
    def __init__(self):
        self.new_urls = set()
        self.old_urls = set()
    
    def add_new_url(self,url):
        if url is None:
            return 
        if url not in self.new_urls and url not in self.old_urls:
            self.new_urls.add(url)
            
    def add_new_urls(self,urls):
        if urls is None or len(urls) == 0:
            return 
        for url in urls:
            self.add_new_url(url)
            
        
    def has_new_url(self):
        return len(self.new_urls) != 0
        
    def get_new_url(self):
        new_url = self.new_urls.pop()
        self.old_urls.add(new_url)
        return new_url
    
    
# 网页下载管理器
class HtmlDownloader(object):
    def download(slef,url):
        if url is None:
            return None
        request = requests.get(url)
        #print('HtmlDownloader:',request.text)
        return request.text
        
# 网页格式管理器
class HtmlParser(object):
    def parse(self,page_url,html_cont):
        if page_url is None or html_cont is None:
            return 
        soup = bfs(html_cont,'html.parser')
        new_urls = self._get_new_urls(page_url,soup)
        new_data = self._get_new_data(page_url,soup)
        #print(new_data[1])
        return new_urls,new_data
    
    def _get_new_urls(self,page_url,soup):
        new_urls = set()
        links = soup.select('.bn_a')
        for link in links:
            new_url = link['href']
            new_full_url = page_url+new_url
            new_urls.add(new_full_url)
        return new_urls
            
                                          
    def _get_new_data(self,page_url,soup):
        res_url=[]
        res_title=[]
        res_summary=[]
        res_url.append(page_url)
        for news in soup.select('.news_li'):
            title_node = news.select('h2')
            for title_n in title_node:
                res_title.append(title_n.select('a')[0].text)
            summary_node = news.select('p')
            for summary_n in summary_node:
                res_summary.append(summary_n.text)
        return res_url,res_title,res_summary
        
# 网页输出 管理器
class HtmlOutputer(object):
    def __init__(self):
        self.datas=[]
    
    def collect_data(self,data):
        if data is None:
            return 
        self.datas.append(data[0])
        self.datas.append(data[1])
        self.datas.append(data[2])
        
    def output_html(self):
        fout = open('output.html','w')
        fout.write('<html>')
        fout.write('<body>')
        fout.write('<table>')
        i = 0 
        for data in self.datas:
            fout.write('<tr>')
            if i%3 == 0:
                fout.write('<th>'+data[0]+'</th>')
            elif i%3 == 1 or i%3 == 2 :
                for msg in data:
                    fout.write('<td>'+msg+'</td>')
            fout.write('</tr>')
            i = i + 1
                
        fout.write('</table>')
        fout.write('</body>')
        fout.write('</html>')

# 爬虫主程序
class SpiderMain(object):
    def __init__(self):
        self.urls = UrlManager()
        self.downloader = HtmlDownloader()
        self.parser = HtmlParser()
        self.outputer = HtmlOutputer()
        
    def craw(self,root_url):
        count = 1 
        self.urls.add_new_url(root_url)
        while self.urls.has_new_url():
            try:
                new_url = self.urls.get_new_url()
                html_cont = self.downloader.download(new_url)
                new_urls,new_data = self.parser.parse(new_url,html_cont)
                self.outputer.collect_data(new_data)

                self.urls.add_new_urls(new_urls)
                print('craw', count,new_url)
                count = count + 1
                if count == 20:
                    break
            except:
                print('craw failed',count,new_url)

        self.outputer.output_html()

    
if __name__=='__main__':
    root_url='https://www.thepaper.cn/'
    obj_spider = SpiderMain()
    obj_spider.craw(root_url)

    

craw 1 https://www.thepaper.cn/
craw 2 https://www.thepaper.cn/channel_26916
craw 3 https://www.thepaper.cn/gov_publish.jsp
craw 4 https://www.thepaper.cn/gov_publish.jspchannel_36079
craw 5 https://www.thepaper.cn/channel_25952
craw 6 https://www.thepaper.cn/channel_25953
craw 7 https://www.thepaper.cn/ask_index.jsp
craw 8 https://www.thepaper.cn/channel_25953channel_25950
craw 9 https://www.thepaper.cn/channel_25952/
craw 10 https://www.thepaper.cn/channel_25952ask_index.jsp
craw 11 https://www.thepaper.cn/channel_25952channel_25951
craw 12 https://www.thepaper.cn/channel_25951
craw 13 https://www.thepaper.cn/channel_26916channel_25953
craw 14 https://www.thepaper.cn/ask_index.jsp/
craw 15 https://www.thepaper.cn/channel_26916channel_36079
craw 16 https://www.thepaper.cn/ask_index.jspchannel_36079
craw 17 https://www.thepaper.cn/channel_25952channel_25953
craw 18 https://www.thepaper.cn/gov_publish.jsp/
craw 19 https://www.thepaper.cn/channel_25951channel_36079


## 爬去 17k小说

In [1]:
import requests
import re
shuhao=str(input('请输入数号：'))
url='http://www.17k.com/list/'+shuhao+'.html'
r=requests.get(url)
r.encoding="utf-8"
#print(r.text)
tt=re.findall(r'<h1 class="Title">(.*?)</h1>',r.text)
zhanjie=re.findall(r'<a target="_blank" href="(.*?)" title=',r.text)
shu_ming=re.findall(r'<meta name="Keywords" content="(.*?),',r.text)
if len(tt)==0:exit('错误的书号')
print('网址：'+url)
print('小说将保存到代码所在目录！')
print('正在爬取小说：《%s》'%shu_ming[0])
fb=open('%s.txt'%tt,'w',encoding='utf-8')
fb.write(shu_ming[0])
for m in zhanjie:
    #print(m)
    m='http://www.17k.com/'+m
    #print(m)
    sxnr=requests.get(m)
    sxnr.encoding = 'utf-8'
    #print(sxnr.text)
    sxzw=re.findall(r'<div class="p">([\s\S]*?)<div class="author-say"></div>',sxnr.text)[0]
    sxzj=re.findall(r'<h1>([\s\S]*?)</h1>',sxnr.text)[0]
    sxzw=sxzw.replace('\n&#12288;&#12288;','')
    sxzw = sxzw.replace('<br />','\n')
    sxzw = sxzw.replace('&#12288;&#12288;', '')
    sxzw = sxzw.replace('&quot;', '')
    fb.write(sxzj)
    fb.write('\n')
    fb.write(sxzw)
    fb.write('\n')
    fb.write('\n')
    #print(sxzj)
    print('>',end='')
print('爬取完毕！')
fb.close()


请输入数号：45
网址：http://www.17k.com/list/45.html
小说将保存到代码所在目录！


IndexError: list index out of range

## Git Test

In [2]:
#  git add1